#### This file demonstrates the mode selection process, and is used to populate table X in paper. 

In [ ]:
import numpy as np
import json
import bgp_qnm_fits as bgp
import time

SIM_ID = "0010"
DATA_TYPE = 'news'
N_MAX = 6
L_MAX = 7

SPHERICAL_MODES_PE = [(2, 2)]

INITIAL_MODES = [] #[(*s, 0, 1) for s in SPHERICAL_MODES_PE]

CANDIDATE_MODES = [(*s, n, 1) for s in SPHERICAL_MODES_PE for n in range(0, N_MAX + 1)] + \
                   [(*s, n, -1) for s in SPHERICAL_MODES_PE for n in range(0, N_MAX + 1)] 

THRESHOLD = 0.9999
T0 = 30
T = 100
N_DRAWS = 1000
INCLUDE_CHIF = False
INCLUDE_MF = False

log_threshold = np.log(THRESHOLD)

In [43]:
tuned_param_dict_GP = bgp.get_tuned_param_dict("GP", data_type=DATA_TYPE)[SIM_ID]
sim = bgp.SXS_CCE(SIM_ID, type=DATA_TYPE, lev="Lev5", radius="R2")

In [44]:
select_object = bgp.BGP_select(
            sim.times,
            sim.h,
            INITIAL_MODES,
            sim.Mf,
            sim.chif_mag,
            tuned_param_dict_GP,
            bgp.kernel_GP,
            t0=T0,
            candidate_modes=CANDIDATE_MODES,
            log_threshold=log_threshold,
            candidate_type="all",
            num_draws=N_DRAWS,
            T=T,
            spherical_modes=SPHERICAL_MODES_PE,
            include_chif=INCLUDE_CHIF,
            include_Mf=INCLUDE_MF,
            data_type=DATA_TYPE
        )

Adding mode (2, 2, 0, 1) with significance 1.0.
Adding mode (3, 3, 0, 1) with significance 1.0.
Adding mode (4, 4, 0, 1) with significance 1.0.
Adding mode (2, 2, 0, 1, 2, 2, 0, 1) with significance 1.0.
Adding mode (2, 2, 1, 1) with significance 1.0.
Adding mode (3, 3, 1, 1) with significance 1.0.
Adding mode (2, 2, 2, 1) with significance 1.0.
Adding mode (2, 2, 3, 1) with significance 1.0.
Adding mode (2, 2, 4, 1) with significance 1.0.
Adding mode (2, 2, 5, 1) with significance 1.0.
Adding mode (2, 2, 6, 1) with significance 1.0.
Adding mode (4, 4, 1, 1) with significance 1.0.
Stopping: no more significant modes
Next mode is (4, 4, 2, 1) with log significance -0.00046554406891514066
Final mode content [(2, 2, 0, 1), (3, 3, 0, 1), (4, 4, 0, 1), (2, 2, 0, 1, 2, 2, 0, 1), (2, 2, 1, 1), (3, 3, 1, 1), (2, 2, 2, 1), (2, 2, 3, 1), (2, 2, 4, 1), (2, 2, 5, 1), (2, 2, 6, 1), (4, 4, 1, 1)]


In [45]:
np.exp(-0.0001921868425761782)

0.9998078316241321

In [46]:
modes_lists = select_object.full_modes
dot_products = select_object.dot_products

In [ ]:
mismatch_22 = []
mismatch_32 = []
mismatch_44 = []

for i, mode in enumerate(modes_lists):

    model = modes_lists[:i+1]

    fit = bgp.BGP_fit(sim.times, 
                        sim.h, 
                        model, 
                        sim.Mf, 
                        sim.chif_mag, 
                        tuned_param_dict_GP, 
                        bgp.kernel_GP, 
                        t0=T0, 
                        T=T, 
                        decay_corrected=True,
                        spherical_modes = SPHERICAL_MODES_PE,
                        include_chif=INCLUDE_CHIF,
                        include_Mf=INCLUDE_MF,
                        data_type=DATA_TYPE)

    mismatch_22.append(bgp.mismatch(fit.fit["model_array_linear"][0:1], fit.fit["data_array_masked"][0:1], fit.fit["noise_covariance"][0:1]))
    mismatch_32.append(bgp.mismatch(fit.fit["model_array_linear"][1:2], fit.fit["data_array_masked"][1:2], fit.fit["noise_covariance"][1:2]))
    mismatch_44.append(bgp.mismatch(fit.fit["model_array_linear"][2:3], fit.fit["data_array_masked"][2:3], fit.fit["noise_covariance"][2:3]))


In [ ]:
import pandas as pd
# Prepare table data
table_data = []
for i, (mode, dot, m22, m32, m44) in enumerate(zip(modes_lists, dot_products, mismatch_22, mismatch_32, mismatch_44), 1):
    sign = "+" if mode[3] == 1 else "-"
    if len(mode) == 8:
        mode_name = f"({mode[0]}, {mode[1]}, {mode[2]}, {sign})^2"
    else:
        mode_name = f"({mode[0]}, {mode[1]}, {mode[2]}, {sign})"
    table_data.append({
        'Iteration': i,
        'Mode Added': mode_name,
        'Dot Product': dot,
        'Mismatch 22': m22,
        'Mismatch 32': m32,
        'Mismatch 44': m44
    })

# Pandas styled table
df = pd.DataFrame(table_data)
display(df.style.format({
    'Dot Product': '{:.3g}',
    'Mismatch 22': '{:.3g}',
    'Mismatch 32': '{:.3g}',
    'Mismatch 44': '{:.3g}'
}))

,Iteration,Mode Added,Dot Product,Mismatch 22,Mismatch 32,Mismatch 44
0,1,"(2, 2, 0, +)",1.35e+07,0.00019,nan,nan
1,2,"(3, 3, 0, +)",8.32e+06,0.00019,3.03e-05,nan
2,3,"(4, 4, 0, +)",3.83e+06,0.00019,3.03e-05,0.00024
3,4,"(2, 2, 0, +)^2",1.62e+03,0.00019,3.03e-05,2.8e-05
4,5,"(2, 2, 1, +)",1.26e+03,0.000143,3.03e-05,2.8e-05
5,6,"(3, 3, 1, +)",384,0.000143,7.17e-06,2.8e-05
6,7,"(2, 2, 2, +)",194,0.000136,7.17e-06,2.8e-05
7,8,"(2, 2, 3, +)",158,0.00013,7.17e-06,2.8e-05
8,9,"(2, 2, 4, +)",131,0.000126,7.17e-06,2.8e-05
9,10,"(2, 2, 5, +)",108,0.000122,7.17e-06,2.8e-05


In [ ]:
import numpy as np
# Plain text for LaTeX with Significance column, three significant figures, scientific notation
print('%')
for row in table_data:
    dot_product = f"{row['Dot Product']/2:.3g}"
    m22 = f"{float(row['Mismatch 22']):.3g}"
    m32 = f"{float(row['Mismatch 32']):.3g}"
    m44 = f"{float(row['Mismatch 44']):.3g}"
    dot_product_sci = f"{float(dot_product):.2e}"
    base, exp = dot_product_sci.split('e')
    dot_product_tex = f"{base}\\times 10^{{{int(exp)}}}"
    significance = f"1 - \exp(-{dot_product_tex}) \\approx {1 - np.exp(-float(dot_product)):.3g}"
    print(f"{row['Iteration']} & ${row['Mode Added']}$ & ${significance}$ & {m22} & {m32} & {m44} \\\ %")

%
1 & $(2, 2, 0, +)$ & $1 - \exp(-6.75\times 10^{6}) \approx 1$ & 0.00019 & nan & nan \\ %
2 & $(3, 3, 0, +)$ & $1 - \exp(-4.16\times 10^{6}) \approx 1$ & 0.00019 & 3.03e-05 & nan \\ %
3 & $(4, 4, 0, +)$ & $1 - \exp(-1.91\times 10^{6}) \approx 1$ & 0.00019 & 3.03e-05 & 0.00024 \\ %
4 & $(2, 2, 0, +)^2$ & $1 - \exp(-8.11\times 10^{2}) \approx 1$ & 0.00019 & 3.03e-05 & 2.8e-05 \\ %
5 & $(2, 2, 1, +)$ & $1 - \exp(-6.28\times 10^{2}) \approx 1$ & 0.000143 & 3.03e-05 & 2.8e-05 \\ %
6 & $(3, 3, 1, +)$ & $1 - \exp(-1.92\times 10^{2}) \approx 1$ & 0.000143 & 7.17e-06 & 2.8e-05 \\ %
7 & $(2, 2, 2, +)$ & $1 - \exp(-9.68\times 10^{1}) \approx 1$ & 0.000136 & 7.17e-06 & 2.8e-05 \\ %
8 & $(2, 2, 3, +)$ & $1 - \exp(-7.92\times 10^{1}) \approx 1$ & 0.00013 & 7.17e-06 & 2.8e-05 \\ %
9 & $(2, 2, 4, +)$ & $1 - \exp(-6.54\times 10^{1}) \approx 1$ & 0.000126 & 7.17e-06 & 2.8e-05 \\ %
10 & $(2, 2, 5, +)$ & $1 - \exp(-5.42\times 10^{1}) \approx 1$ & 0.000122 & 7.17e-06 & 2.8e-05 \\ %
11 & $(2, 2, 6, +)$ & $